In [1]:
import pandas as pd
import numpy as np

import pickle

# gensim
from gensim import corpora, models, similarities, matutils

# sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import nltk
import spacy

First, pulling in modgroups. Contains ~296k unique names and mod classes. Will append this value to the final data

In [2]:
mod_groups = pd.read_csv('/Users/robertpagano/Desktop/AOC final graphs/Mod_class_tucker.csv')

In [3]:
with open('/Users/robertpagano/metis_data/project_5/final_raw_tucker_df.pickle', 'rb') as f:
    df_tuck = pickle.load(f)

In [4]:
df_tuck.head()

,tweet_id,user_id,display_name,screen_name,tweet_date,main_text,fav_count,rt_count,followers_count,is_verified,...,retweet_fav_count,retweet_followers_count,quoted_text,quoted_id,quoted_user_id,quoted_screen_name,quoted_is_verified,quoted_fav_count,is_retweet,is_quote
0,1108898530125668352,819975326663114754,Sheryl,Sheryl2100,Fri Mar 22 01:09:46 +0000 2019,RT @JuddLegum: .@progressive quietly stopped a...,0,535,220,False,...,1035.0,337485.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0
1,1108898518821933056,1908173622,❌Yoo Hoo ✝️ Ecc 10:2 🇺🇸 Pro 15:3,Watchman4the1,Fri Mar 22 01:09:43 +0000 2019,@TuckerCarlson &amp; @seanhannity&gt;&gt;&gt; ...,0,0,3440,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,1108898513805627397,578116913,Samuel Adams,jgrissom63,Fri Mar 22 01:09:42 +0000 2019,@marklevinshow Sorry but like thousands others...,0,0,2568,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,1108898512798957568,3298861641,राधिका,nakiarn25,Fri Mar 22 01:09:42 +0000 2019,RT @JuddLegum: .@progressive quietly stopped a...,0,535,62,False,...,1035.0,337485.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0
4,1108898512173977605,2425284969,shaivon Cherry,sherryw329,Fri Mar 22 01:09:42 +0000 2019,RT @JuddLegum: .@progressive quietly stopped a...,0,535,492,False,...,1035.0,337485.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0


In [5]:
df_tuck.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 509602 entries, 0 to 3677
Data columns (total 28 columns):
tweet_id                   509602 non-null int64
user_id                    509602 non-null int64
display_name               509602 non-null object
screen_name                509602 non-null object
tweet_date                 509602 non-null object
main_text                  509602 non-null object
fav_count                  509602 non-null int64
rt_count                   509602 non-null int64
followers_count            509602 non-null int64
is_verified                509602 non-null bool
reply_to_tweet_id          89684 non-null float64
reply_to_user_id           104241 non-null float64
reply_to_screen_name       104241 non-null object
retweet_text               359957 non-null object
retweet_id                 359957 non-null float64
retweet_user_id            359957 non-null float64
retweet_screen_name        359957 non-null object
retweet_is_verified        359957 non-null ob

In [6]:
df_tuck['screen_name'].nunique()

214060

In [7]:
mod_groups['Id'].nunique()

161593

Not sure why, but we must have cut out around 150k names from the network data. Should probably figure out who that is before continuing, I don't want to not include important accounts

In [8]:
mod_groups['modularity_class'].nunique()

13

In [9]:
mod_groups.groupby(['modularity_class']).count()[['Id']]

,Id
modularity_class,
0,18934
1,8579
2,41051
3,29213
4,1805
5,7669
6,1567
7,34342
8,1734


So let's look at ALL communities, and try to find 3 from each side
 
So we need to find the top accounts in each group, do NLP analysis of each group (topic modeling / sentiment analysis)


Next, to check - append community to each account. Then make sure the top x accounts have one of these communties

In [10]:
mod_groups.head()

,Id,Label,timeset,modularity_class,Weighted Degree
0,Sheryl2100,Sheryl2100,NaN,7,5
1,JuddLegum,JuddLegum,NaN,0,2737
2,nakiarn25,nakiarn25,NaN,0,1
3,sherryw329,sherryw329,NaN,11,9
4,bbgomex17,bbgomex17,NaN,0,3


In [11]:
mod_groups.drop(columns=['Id', 'timeset'], inplace=True)
mod_groups.head()

,Label,modularity_class,Weighted Degree
0,Sheryl2100,7,5
1,JuddLegum,0,2737
2,nakiarn25,0,1
3,sherryw329,11,9
4,bbgomex17,0,3


# TO DO

1. Append community to DF
2. Check top accounts are a part of community (and if they are, look back to see who we lost)
3. Topic modeling with LSA / NMF. Get top words each topic, etc.
4. New Jupyter Notebook - sentiment analysis on each topic by community

In [12]:
df_tuck = pd.merge(df_tuck, mod_groups, how='left', left_on='screen_name', right_on='Label')
df_tuck.head()

,tweet_id,user_id,display_name,screen_name,tweet_date,main_text,fav_count,rt_count,followers_count,is_verified,...,quoted_id,quoted_user_id,quoted_screen_name,quoted_is_verified,quoted_fav_count,is_retweet,is_quote,Label,modularity_class,Weighted Degree
0,1108898530125668352,819975326663114754,Sheryl,Sheryl2100,Fri Mar 22 01:09:46 +0000 2019,RT @JuddLegum: .@progressive quietly stopped a...,0,535,220,False,...,NaN,NaN,NaN,NaN,NaN,1,0,Sheryl2100,7.0,5.0
1,1108898518821933056,1908173622,❌Yoo Hoo ✝️ Ecc 10:2 🇺🇸 Pro 15:3,Watchman4the1,Fri Mar 22 01:09:43 +0000 2019,@TuckerCarlson &amp; @seanhannity&gt;&gt;&gt; ...,0,0,3440,False,...,NaN,NaN,NaN,NaN,NaN,0,0,Watchman4the1,2.0,2.0
2,1108898513805627397,578116913,Samuel Adams,jgrissom63,Fri Mar 22 01:09:42 +0000 2019,@marklevinshow Sorry but like thousands others...,0,0,2568,False,...,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN
3,1108898512798957568,3298861641,राधिका,nakiarn25,Fri Mar 22 01:09:42 +0000 2019,RT @JuddLegum: .@progressive quietly stopped a...,0,535,62,False,...,NaN,NaN,NaN,NaN,NaN,1,0,nakiarn25,0.0,1.0
4,1108898512173977605,2425284969,shaivon Cherry,sherryw329,Fri Mar 22 01:09:42 +0000 2019,RT @JuddLegum: .@progressive quietly stopped a...,0,535,492,False,...,NaN,NaN,NaN,NaN,NaN,1,0,sherryw329,11.0,9.0


In [13]:
pd.set_option('display.max_columns', 500)

In [14]:
df_tuck.head()

,tweet_id,user_id,display_name,screen_name,tweet_date,main_text,fav_count,rt_count,followers_count,is_verified,reply_to_tweet_id,reply_to_user_id,reply_to_screen_name,retweet_text,retweet_id,retweet_user_id,retweet_screen_name,retweet_is_verified,retweet_fav_count,retweet_followers_count,quoted_text,quoted_id,quoted_user_id,quoted_screen_name,quoted_is_verified,quoted_fav_count,is_retweet,is_quote,Label,modularity_class,Weighted Degree
0,1108898530125668352,819975326663114754,Sheryl,Sheryl2100,Fri Mar 22 01:09:46 +0000 2019,RT @JuddLegum: .@progressive quietly stopped a...,0,535,220,False,NaN,NaN,None,.@progressive quietly stopped advertising on T...,1.108893e+18,15464697.0,JuddLegum,True,1035.0,337485.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,Sheryl2100,7.0,5.0
1,1108898518821933056,1908173622,❌Yoo Hoo ✝️ Ecc 10:2 🇺🇸 Pro 15:3,Watchman4the1,Fri Mar 22 01:09:43 +0000 2019,@TuckerCarlson &amp; @seanhannity&gt;&gt;&gt; ...,0,0,3440,False,NaN,22703645.0,TuckerCarlson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,Watchman4the1,2.0,2.0
2,1108898513805627397,578116913,Samuel Adams,jgrissom63,Fri Mar 22 01:09:42 +0000 2019,@marklevinshow Sorry but like thousands others...,0,0,2568,False,1.108851e+18,38495835.0,marklevinshow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN
3,1108898512798957568,3298861641,राधिका,nakiarn25,Fri Mar 22 01:09:42 +0000 2019,RT @JuddLegum: .@progressive quietly stopped a...,0,535,62,False,NaN,NaN,None,.@progressive quietly stopped advertising on T...,1.108893e+18,15464697.0,JuddLegum,True,1035.0,337485.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,nakiarn25,0.0,1.0
4,1108898512173977605,2425284969,shaivon Cherry,sherryw329,Fri Mar 22 01:09:42 +0000 2019,RT @JuddLegum: .@progressive quietly stopped a...,0,535,492,False,NaN,NaN,None,.@progressive quietly stopped advertising on T...,1.108893e+18,15464697.0,JuddLegum,True,1035.0,337485.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,sherryw329,11.0,9.0


Now I want to put a counter for each screen name as a row

In [15]:
df_tuck['screenname_freq'] = df_tuck.groupby('screen_name')['screen_name'].transform('count')

In [16]:
df_tuck.head()

,tweet_id,user_id,display_name,screen_name,tweet_date,main_text,fav_count,rt_count,followers_count,is_verified,reply_to_tweet_id,reply_to_user_id,reply_to_screen_name,retweet_text,retweet_id,retweet_user_id,retweet_screen_name,retweet_is_verified,retweet_fav_count,retweet_followers_count,quoted_text,quoted_id,quoted_user_id,quoted_screen_name,quoted_is_verified,quoted_fav_count,is_retweet,is_quote,Label,modularity_class,Weighted Degree,screenname_freq
0,1108898530125668352,819975326663114754,Sheryl,Sheryl2100,Fri Mar 22 01:09:46 +0000 2019,RT @JuddLegum: .@progressive quietly stopped a...,0,535,220,False,NaN,NaN,None,.@progressive quietly stopped advertising on T...,1.108893e+18,15464697.0,JuddLegum,True,1035.0,337485.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,Sheryl2100,7.0,5.0,7
1,1108898518821933056,1908173622,❌Yoo Hoo ✝️ Ecc 10:2 🇺🇸 Pro 15:3,Watchman4the1,Fri Mar 22 01:09:43 +0000 2019,@TuckerCarlson &amp; @seanhannity&gt;&gt;&gt; ...,0,0,3440,False,NaN,22703645.0,TuckerCarlson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,Watchman4the1,2.0,2.0,18
2,1108898513805627397,578116913,Samuel Adams,jgrissom63,Fri Mar 22 01:09:42 +0000 2019,@marklevinshow Sorry but like thousands others...,0,0,2568,False,1.108851e+18,38495835.0,marklevinshow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,2
3,1108898512798957568,3298861641,राधिका,nakiarn25,Fri Mar 22 01:09:42 +0000 2019,RT @JuddLegum: .@progressive quietly stopped a...,0,535,62,False,NaN,NaN,None,.@progressive quietly stopped advertising on T...,1.108893e+18,15464697.0,JuddLegum,True,1035.0,337485.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,nakiarn25,0.0,1.0,1
4,1108898512173977605,2425284969,shaivon Cherry,sherryw329,Fri Mar 22 01:09:42 +0000 2019,RT @JuddLegum: .@progressive quietly stopped a...,0,535,492,False,NaN,NaN,None,.@progressive quietly stopped advertising on T...,1.108893e+18,15464697.0,JuddLegum,True,1035.0,337485.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,sherryw329,11.0,9.0,11


In [17]:
df_no_class = df_tuck[df_tuck['modularity_class'].isnull() == True]

In [18]:
df_no_class.sort_values('followers_count', ascending=False).head(20)

,tweet_id,user_id,display_name,screen_name,tweet_date,main_text,fav_count,rt_count,followers_count,is_verified,reply_to_tweet_id,reply_to_user_id,reply_to_screen_name,retweet_text,retweet_id,retweet_user_id,retweet_screen_name,retweet_is_verified,retweet_fav_count,retweet_followers_count,quoted_text,quoted_id,quoted_user_id,quoted_screen_name,quoted_is_verified,quoted_fav_count,is_retweet,is_quote,Label,modularity_class,Weighted Degree,screenname_freq
360334,1105880340596289537,5695632,BuzzFeed,BuzzFeed,Wed Mar 13 17:16:34 +0000 2019,"""Catastrophe"" Stars Rob Delaney And Sharon Hor...",26,2,6525630,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1
22324,1108409720803655680,4532891,MUCH,Much,Wed Mar 20 16:47:25 +0000 2019,#Tosh returned last night as Daniel took aim a...,5,1,973390,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1
125089,1107320457185947649,9763482,New York Daily News,NYDailyNews,Sun Mar 17 16:39:04 +0000 2019,Trump contended that Fox News had caved under ...,6,2,689764,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1
362555,1105876631611211778,30205586,Adweek,Adweek,Wed Mar 13 17:01:49 +0000 2019,Fox News stands with Tucker Carlson and Jeanin...,5,0,614737,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,3
189170,1106601459934728193,30205586,Adweek,Adweek,Fri Mar 15 17:02:02 +0000 2019,Opinion | Boeing and advertisers on YouTube an...,1,0,614718,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,3
113336,1107402231446470656,30205586,Adweek,Adweek,Sun Mar 17 22:04:01 +0000 2019,Some advertisers are starting to distance them...,8,2,614717,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,3
121650,1107344107725193216,156651229,Guardian World,guardianworld,Sun Mar 17 18:13:03 +0000 2019,'Bring back Jeanine Pirro': Trump defends Fox ...,3,3,497185,False,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1
192441,1106570510224244736,14602259,Boston.com,BostonDotCom,Fri Mar 15 14:59:03 +0000 2019,Tucker Carlson abandons plan for Fox News stud...,3,0,460663,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1
421680,1105776788972728320,14957318,NBC Chicago,nbcchicago,Wed Mar 13 10:25:05 +0000 2019,"Fox News advertisers pause on Jeanine Pirro, T...",2,0,431235,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1
232069,1106243981657165835,14221917,Philly.com,phillydotcom,Thu Mar 14 17:21:32 +0000 2019,"Tucker Carlson shouldn’t apologize, even in th...",4,0,384631,True,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,1


In [19]:
df_no_class.shape

(79489, 32)

In [20]:
df_tuck.shape

(509602, 32)

In [21]:
509602-79489

430113

So it looks like the network communities just ignored those with very low frequency in the dataset, which is fine. If I ignore those without a modularity class, I end up with roughly 1.3 million observations (removing 307k). This should be fine for NLP analysis, so I'll just remove those

In [22]:
df_tuck = df_tuck[df_tuck['modularity_class'].isnull() == False]

In [23]:
df_tuck.shape
## And I'll pickle it

(430113, 32)

In [24]:
with open('/Users/robertpagano/metis_data/project_5/tuck_nlp_null_class_removed.pickle', 'wb') as to_write:
    pickle.dump(df_tuck, to_write)

Now filter for just communties with enough data, and find the top posters in each

0
1 (maybe not)
2
3
4
5
6
7
8
9
10
11 (maybe not)
26
40

In [25]:
# list of communities to run NLP on
list_of_coms = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [26]:
df_tuck_coms = df_tuck[df_tuck['modularity_class'].isin(list_of_coms)]

In [30]:
df_tuck_coms.shape

(430113, 32)

In [31]:
# get counts of tweets by modularity class
df_tuck_coms.groupby(['modularity_class']).count()[['screen_name']]

,screen_name
modularity_class,
0.0,45084
1.0,16631
2.0,136775
3.0,78732
4.0,3549
5.0,10017
6.0,2138
7.0,87970
8.0,4429


 - append retweet_mod_class to all retweets?
 - Append pagerank to each tweet, so I can get most influential nodes by class
 - sort by follower count first, then come back and add pagerank, betweenness, etc.

In [74]:
# So basically I need to do a groupby, screenname, follower count, head(10), and functionize it

In [32]:
df_tuck_coms.columns

Index(['tweet_id', 'user_id', 'display_name', 'screen_name', 'tweet_date',
       'main_text', 'fav_count', 'rt_count', 'followers_count', 'is_verified',
       'reply_to_tweet_id', 'reply_to_user_id', 'reply_to_screen_name',
       'retweet_text', 'retweet_id', 'retweet_user_id', 'retweet_screen_name',
       'retweet_is_verified', 'retweet_fav_count', 'retweet_followers_count',
       'quoted_text', 'quoted_id', 'quoted_user_id', 'quoted_screen_name',
       'quoted_is_verified', 'quoted_fav_count', 'is_retweet', 'is_quote',
       'Label', 'modularity_class', 'Weighted Degree', 'screenname_freq'],
      dtype='object')

I'm going to add pagerank here, and then add it to the "raw" file, and then re-run all the separate communities things

In [113]:
with open('/Users/robertpagano/src/metis_project_kojak/network_files/aoc/aoc_pagerank_all.pickle', 'rb') as f:
    pr = pickle.load(f)

In [114]:
df_aoc_coms['PageRank'] = df_aoc_coms['screen_name'].map(pr)
df_aoc_coms.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,tweet_id,user_id,display_name,screen_name,tweet_date,main_text,fav_count,rt_count,followers_count,is_verified,reply_to_tweet_id,reply_to_user_id,reply_to_screen_name,retweet_text,retweet_id,retweet_user_id,retweet_screen_name,retweet_is_verified,retweet_fav_count,retweet_followers_count,quoted_text,quoted_id,quoted_user_id,quoted_screen_name,quoted_is_verified,quoted_fav_count,is_retweet,is_quote,Label,modularity_class,screenname_freq,PageRank
0,1105328989005000704,955685794433318912,mythos55,Sandra71005599,Tue Mar 12 04:45:41 +0000 2019,"RT @ARmastrangelo: .@AOC, my entire family imm...",0,5155,232,False,NaN,NaN,None,".@AOC, my entire family immigrated to the Unit...",1.105223e+18,2.652095e+07,ARmastrangelo,True,16041.0,180619.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,Sandra71005599,4.0,13,0.000003
1,1105328986823962625,1523921276,Webcams,webcamsla,Tue Mar 12 04:45:41 +0000 2019,RT @gatewaypundit: Figures: Alexandria Ocasio-...,0,82,1034,False,NaN,NaN,None,Figures: Alexandria Ocasio-Cortez is All Smile...,1.105316e+18,1.921155e+07,gatewaypundit,True,118.0,137906.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,webcamsla,5.0,56,0.000011
2,1105328986219929600,730127113261449217,love trump,DGlavine,Tue Mar 12 04:45:40 +0000 2019,"RT @Spring_Harbinge: 🚨hey, @AOC , Is it true y...",0,178,1064,False,NaN,NaN,None,"🚨hey, @AOC , Is it true your failed children's...",1.105232e+18,9.695692e+17,Spring_Harbinge,False,175.0,34404.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,DGlavine,9.0,8,0.000003
4,1105328984030621697,891939067,Kate B,kateb722,Tue Mar 12 04:45:40 +0000 2019,RT @Education4Libs: SHARE THIS EVERYWHERE befo...,0,704,7334,False,NaN,NaN,None,SHARE THIS EVERYWHERE before YouTube takes it ...,1.105315e+18,8.176611e+17,Education4Libs,False,833.0,349574.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,kateb722,9.0,13,0.000003
5,1105328982562664448,274207288,Drayton,ScoutDrayton,Tue Mar 12 04:45:40 +0000 2019,RT @NoahPollak: AOC and Ilhan Omar's good frie...,0,1088,11072,False,NaN,NaN,None,AOC and Ilhan Omar's good friend Rep. Rashida ...,1.105199e+18,1.067114e+08,NoahPollak,False,1619.0,23822.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,ScoutDrayton,5.0,56,0.000012


In [129]:
df_coms_raw = df_aoc_coms.loc[:, ['screen_name', 'followers_count', 'modularity_class', 'main_text', 'retweet_text', 'tweet_date', 'PageRank', 'rt_count']]

In [130]:
df_com_0 = df_coms_raw[df_coms_raw['modularity_class'] == 0]
df_com_1 = df_coms_raw[df_coms_raw['modularity_class'] == 1]
df_com_2 = df_coms_raw[df_coms_raw['modularity_class'] == 2]
df_com_3 = df_coms_raw[df_coms_raw['modularity_class'] == 3]
df_com_4 = df_coms_raw[df_coms_raw['modularity_class'] == 4]
df_com_5 = df_coms_raw[df_coms_raw['modularity_class'] == 5]
df_com_6 = df_coms_raw[df_coms_raw['modularity_class'] == 6]
df_com_7 = df_coms_raw[df_coms_raw['modularity_class'] == 7]
df_com_8 = df_coms_raw[df_coms_raw['modularity_class'] == 8]
df_com_9 = df_coms_raw[df_coms_raw['modularity_class'] == 9]
df_com_10 = df_coms_raw[df_coms_raw['modularity_class'] == 10]
df_com_11 = df_coms_raw[df_coms_raw['modularity_class'] == 11]
df_com_26 = df_coms_raw[df_coms_raw['modularity_class'] == 26]
df_com_40 = df_coms_raw[df_coms_raw['modularity_class'] == 40]


In [131]:
print(df_com_0.shape)
print(df_com_1.shape)
print(df_com_2.shape)
print(df_com_3.shape)
print(df_com_4.shape)
print(df_com_5.shape)
print(df_com_6.shape)
print(df_com_7.shape)
print(df_com_8.shape)
print(df_com_9.shape)
print(df_com_10.shape)
print(df_com_11.shape)
print(df_com_26.shape)
print(df_com_40.shape)

(34997, 8)
(1446, 8)
(8345, 8)
(9346, 8)
(224793, 8)
(237746, 8)
(190998, 8)
(135612, 8)
(8192, 8)
(425343, 8)
(13157, 8)
(2270, 8)
(1141, 8)
(2628, 8)


In [132]:
df_com_0 = df_com_0.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_1 = df_com_1.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_2 = df_com_2.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_3 = df_com_3.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_4 = df_com_4.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_5 = df_com_5.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_6 = df_com_6.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_7 = df_com_7.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_8 = df_com_8.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_9 = df_com_9.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_10 = df_com_10.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_11 = df_com_11.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_26 = df_com_26.sort_values(by=['PageRank', 'rt_count'], ascending=False)
df_com_40 = df_com_40.sort_values(by=['PageRank', 'rt_count'], ascending=False)

In [150]:
df_com_0.head()

,screen_name,followers_count,modularity_class,main_text,retweet_text,tweet_date,PageRank,rt_count
405812,DavidJHarrisJr,75631,0.0,So @aoc answered a casting call... and now she...,NaN,Mon Mar 11 01:16:04 +0000 2019,0.004242,6936
712255,DavidJHarrisJr,75652,0.0,So @aoc wants to raise our taxes and she hasn’...,NaN,Sun Mar 10 03:49:44 +0000 2019,0.004242,1127
465115,DavidJHarrisJr,75638,0.0,Now @aoc says America is garbage? \nWho electe...,NaN,Sun Mar 10 22:30:39 +0000 2019,0.004242,1056
62994,DavidJHarrisJr,75620,0.0,Former FEC Commissioner says there’s more than...,NaN,Tue Mar 12 01:25:16 +0000 2019,0.004242,997
1205837,DavidJHarrisJr,75690,0.0,Hey AOC! Guess what? Climate change and global...,NaN,Fri Mar 08 05:02:07 +0000 2019,0.004242,838


For the below - I want to create this with just a list of the most influential 20 in each category, for EDA purposes

In [153]:
df_com_0_top_20 = df_com_0.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_1_top_20 = df_com_1.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_2_top_20 = df_com_2.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_3_top_20 = df_com_3.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_4_top_20 = df_com_4.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_5_top_20 = df_com_5.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_6_top_20 = df_com_6.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_7_top_20 = df_com_7.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_8_top_20 = df_com_8.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_9_top_20 = df_com_9.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_10_top_20 = df_com_10.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_11_top_20 = df_com_11.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_26_top_20 = df_com_26.drop_duplicates(subset='screen_name', keep='first').head(20)
df_com_40_top_20 = df_com_40.drop_duplicates(subset='screen_name', keep='first').head(20)

In [1]:
df_com_0_top_20

NameError: name 'df_com_0_top_20' is not defined

Pickling all of my community partitioned data, will begin work on topic modeling in new notebook

In [164]:
with open('/Users/robertpagano/src/metis_project_kojak/network_files/aoc/community_pickles_pre_nlp/df_0.pickle', 'wb') as to_write:
    pickle.dump(df_com_0, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/aoc/community_pickles_pre_nlp/df_1.pickle', 'wb') as to_write:
    pickle.dump(df_com_1, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/aoc/community_pickles_pre_nlp/df_2.pickle', 'wb') as to_write:
    pickle.dump(df_com_2, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/aoc/community_pickles_pre_nlp/df_3.pickle', 'wb') as to_write:
    pickle.dump(df_com_3, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/aoc/community_pickles_pre_nlp/df_4.pickle', 'wb') as to_write:
    pickle.dump(df_com_4, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/aoc/community_pickles_pre_nlp/df_5.pickle', 'wb') as to_write:
    pickle.dump(df_com_5, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/aoc/community_pickles_pre_nlp/df_6.pickle', 'wb') as to_write:
    pickle.dump(df_com_6, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/aoc/community_pickles_pre_nlp/df_7.pickle', 'wb') as to_write:
    pickle.dump(df_com_7, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/aoc/community_pickles_pre_nlp/df_8.pickle', 'wb') as to_write:
    pickle.dump(df_com_8, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/aoc/community_pickles_pre_nlp/df_9.pickle', 'wb') as to_write:
    pickle.dump(df_com_9, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/aoc/community_pickles_pre_nlp/df_10.pickle', 'wb') as to_write:
    pickle.dump(df_com_10, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/aoc/community_pickles_pre_nlp/df_11.pickle', 'wb') as to_write:
    pickle.dump(df_com_11, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/aoc/community_pickles_pre_nlp/df_26.pickle', 'wb') as to_write:
    pickle.dump(df_com_26, to_write)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/aoc/community_pickles_pre_nlp/df_40.pickle', 'wb') as to_write:
    pickle.dump(df_com_40, to_write)